In [59]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [60]:
# Load the nutrition dataset
file_path = '/content/nutrition_dataset.csv'  # Update with your actual dataset path
df = pd.read_csv(file_path)
print("Dataset loaded.")

# Section 1: Data Inspection
print("Checking for missing values:")
print(df.isnull().sum())  # Checks if any column has missing values
print("\nChecking for duplicate values:")
print(df.duplicated().sum())  # Checks if there are duplicate rows

Dataset loaded.
Checking for missing values:
Age                     0
Gender                  0
Height                  0
Weight                  0
Activity Level          0
Dietary Preference      0
Daily Calorie Target    0
Protein                 0
Carbohydrates           0
Fat                     0
Breakfast Suggestion    0
Lunch Suggestion        0
Dinner Suggestion       0
Snack Suggestion        0
Sugar                   0
Sodium                  0
Calories                0
Fiber                   0
Disease                 0
BMI                     0
dtype: int64

Checking for duplicate values:
0


In [61]:
# Dropping irrelevant columns, adding calculated fields, and reordering columns

# Add new columns based on calculations (e.g., Sugar, Sodium, Calories, Fiber)
df['Sugar'] = df['Carbohydrates'] * 0.5
df['Sodium'] = df['Protein'] * 0.2
df['Calories'] = df['Fat'] * 9 + df['Protein'] * 4 + df['Carbohydrates'] * 4
df['Fiber'] = df['Carbohydrates'] * 0.12

# Define a function to assign diseases based on nutrient criteria
def assign_disease(row):
    diseases = []
    if row['Carbohydrates'] < 150 and row['Sugar'] < 100 and row['Protein'] >= 50:
        diseases.append('Diabetes')
    if row['Sugar'] < 100 and row['Fat'] < 50 and row['Carbohydrates'] < 150:
        diseases.append('Acne')
    if row['Calories'] < 1000 and row['Protein'] > 50 and row['Carbohydrates'] < 150:
        diseases.append('Weight Loss')
    if row['Calories'] > 400 and row['Protein'] > 50 and row['Carbohydrates'] > 120 and row['Fat'] > 30:
        diseases.append('Weight Gain')
    if row['Sodium'] < 50 and row['Fat'] < 50:
        diseases.append('Hypertension')
    if row['Fat'] < 50 and row['Protein'] > 50:
        diseases.append('Heart Disease')
    if row['Protein'] < 80 and row['Sodium'] < 50:
        diseases.append('Kidney Disease')
    if row['Fat'] < 5 and row['Fiber'] > 10:
        diseases.append('Cholesterol')
    return ', '.join(diseases) if diseases else 'Normal'

# Apply disease assignment function
df['Disease'] = df.apply(assign_disease, axis=1)

In [62]:
# Section 3: Data Preprocessing (Encoding and Normalization)
# Label encode categorical columns
label_encoders = {}
categorical_columns = ['Gender', 'Activity Level', 'Dietary Preference', 'Disease']
for column in categorical_columns:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le  # Store encoders for possible inverse transformations

# Standardize numeric columns
scaler = StandardScaler()
numeric_features = ['Age', 'Height', 'Weight', 'Daily Calorie Target', 'Protein', 'Carbohydrates', 'Fat', 'Sugar', 'Sodium', 'Calories', 'Fiber']
df[numeric_features] = scaler.fit_transform(df[numeric_features])

In [63]:
# Section 4: Model Training and Evaluation
X = df.drop(['Breakfast Suggestion', 'Lunch Suggestion', 'Dinner Suggestion', 'Snack Suggestion', 'Disease'], axis=1)
y = df['Disease']

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
rf_model.fit(X_train, y_train)

# Evaluate the model
y_pred_rf = rf_model.predict(X_test)
print("Random Forest Model Accuracy:", accuracy_score(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))

# Display feature importance
feature_importances = pd.Series(rf_model.feature_importances_, index=X.columns).sort_values(ascending=False)
print("Feature Importances:\n", feature_importances)

Random Forest Model Accuracy: 1.0
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         2
           5       1.00      1.00      1.00        77
           6       1.00      1.00      1.00         5
           7       1.00      1.00      1.00        12
           8       1.00      1.00      1.00         4

    accuracy                           1.00       100
   macro avg       1.00      1.00      1.00       100
weighted avg       1.00      1.00      1.00       100

Feature Importances:
 Fat                     0.204660
Protein                 0.184947
Calories                0.169875
Sodium                  0.157681
Daily Calorie Target    0.083504
Fiber                   0.056639
Carbohydrates           0.042628
Sugar                   0.034297
Age                     0.015472
Height                  0.014393
BMI                     0.013552
Weight                  0.012410
Dietary Preference      0.003547
Gender                  0.

In [64]:
# Section 5: Neural Network Training and Evaluation
# Define the neural network model
num_classes = len(np.unique(y_train))
nn_model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')
])

# Compile the model
nn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the neural network
y_train_nn = tf.keras.utils.to_categorical(y_train, num_classes=num_classes)
y_test_nn = tf.keras.utils.to_categorical(y_test, num_classes=num_classes)
history = nn_model.fit(X_train, y_train_nn, epochs=50, batch_size=10, validation_split=0.2)

# Evaluate the neural network
test_loss, test_accuracy = nn_model.evaluate(X_test, y_test_nn)
print("Neural Network Test Accuracy:", test_accuracy)

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.7895 - loss: 1.0734 - val_accuracy: 0.8500 - val_loss: 0.5411
Epoch 2/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8754 - loss: 0.4571 - val_accuracy: 0.8625 - val_loss: 0.4918
Epoch 3/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8615 - loss: 0.4905 - val_accuracy: 0.8750 - val_loss: 0.5100
Epoch 4/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8525 - loss: 0.4551 - val_accuracy: 0.8625 - val_loss: 0.4527
Epoch 5/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8600 - loss: 0.4641 - val_accuracy: 0.8625 - val_loss: 0.4403
Epoch 6/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8705 - loss: 0.4468 - val_accuracy: 0.8625 - val_loss: 0.4253
Epoch 7/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8766 - loss: 0.4326 - val_accuracy: 0.8750 - val_loss: 0.4333
Epoch 8/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9065 - loss: 0.3230 - val_accuracy: 0.8750 - val_loss: 0.4245
Epo

In [66]:
# Section 6: Meal Recommendation System with User Input

# Load original dataset with recommendations
nutrition_data = pd.read_csv(file_path)

# Define health conditions
unique_conditions = ["Weight Loss", "Kidney Disease", "Weight Gain", "Hypertension", "Diabetes", "Acne", "Heart Disease"]

# Meal recommendation function
def recommend_meals(name, age, weight, height, dietary_pref, health_conditions):
    filtered_data = nutrition_data[nutrition_data['Dietary Preference'].str.contains(dietary_pref, case=False, na=False)].copy()

    # Filter by health conditions
    if health_conditions:
        condition_pattern = '|'.join(health_conditions)
        filtered_data = filtered_data[filtered_data['Disease'].str.contains(condition_pattern, case=False, na=False)]

    # Check if the filtered dataset is empty
    if filtered_data.empty:
        display(widgets.HTML(f"Hello {name}! No meals found matching your dietary preference and health conditions."))
        return

    # Calculate BMI and suggest weekly meal plan
    bmi = weight / ((height / 100) ** 2)
    display(widgets.HTML(f"Hello {name}! Your BMI is: {bmi:.2f}"))
    if bmi < 18.5:
        recommendation = "Consider a diet to help with weight gain."
    elif 18.5 <= bmi < 24.9:
        recommendation = "Maintain a balanced diet to stay healthy!"
    else:
        recommendation = "Consider a diet plan for weight management."
    display(widgets.HTML(recommendation))
    display(widgets.HTML("Here's a suggested weekly meal plan based on your preferences and health conditions."))

    # Generate weekly meal plan
    weekly_plan = pd.DataFrame({
        "Day": ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"],
        "Breakfast": [random.choice(filtered_data["Breakfast Suggestion"].dropna().tolist()) for _ in range(7)],
        "Lunch": [random.choice(filtered_data["Lunch Suggestion"].dropna().tolist()) for _ in range(7)],
        "Dinner": [random.choice(filtered_data["Dinner Suggestion"].dropna().tolist()) for _ in range(7)],
        "Snack": [random.choice(filtered_data["Snack Suggestion"].dropna().tolist()) for _ in range(7)]
    })
    display(weekly_plan)

# Input widgets for meal recommendations
name_widget = widgets.Text(description="Name")
age_widget = widgets.IntText(description="Age")
weight_widget = widgets.FloatText(description="Weight (kg)")
height_widget = widgets.FloatText(description="Height (cm)")
dietary_pref_widget = widgets.Dropdown(options=nutrition_data['Dietary Preference'].unique(), description="Dietary Preference")

# Checkboxes for health conditions
health_condition_checkboxes = [widgets.Checkbox(value=False, description=condition) for condition in unique_conditions]

# Button for generating meal plan
button = widgets.Button(description="Generate Meal Plan")

# Display widgets
display(name_widget, age_widget, weight_widget, height_widget, dietary_pref_widget)
for checkbox in health_condition_checkboxes:
    display(checkbox)
display(button)

# Button click function
def on_button_clicked(b):
    name = name_widget.value
    age = age_widget.value
    weight = weight_widget.value
    height = height_widget.value
    dietary_pref = dietary_pref_widget.value
    selected_conditions = [cb.description for cb in health_condition_checkboxes if cb.value]
    recommend_meals(name, age, weight, height, dietary_pref, selected_conditions)

button.on_click(on_button_clicked)

Text(value='', description='Name')

IntText(value=0, description='Age')

FloatText(value=0.0, description='Weight (kg)')

FloatText(value=0.0, description='Height (cm)')

Dropdown(description='Dietary Preference', options=('Omnivore', 'Vegetarian', 'Vegan'), value='Omnivore')

...

Button(description='Generate Meal Plan', style=ButtonStyle())

HTML(value='Hello deepak! Your BMI is: 22.53')

HTML(value='Maintain a balanced diet to stay healthy!')

HTML(value="Here's a suggested weekly meal plan based on your preferences and health conditions.")

,Day,Breakfast,Lunch,Dinner,Snack
0,Monday,Scrambled tofu with spinach and tomato,Lentil soup with a side of whole-wheat bread,Tofu stir-fry with vegetables,Fruit and yogurt
1,Tuesday,Scrambled eggs with whole-wheat toast,Vegetarian chili with brown rice,Lentil pasta with vegetables,Greek yogurt with fruit
2,Wednesday,Oatmeal with fruit and nuts,Black bean burger on a whole-wheat bun,Vegetarian chili with brown rice,Apple with peanut butter
3,Thursday,Tofu scramble with vegetables and whole wheat ...,Lentil soup with whole wheat bread,Vegetable curry with brown rice,Trail mix with nuts and dried fruit
4,Friday,Scrambled eggs with whole-wheat toast,Lentil soup with whole wheat bread,Tofu and vegetable stir-fry with brown rice,Trail mix with nuts and dried fruit
5,Saturday,Scrambled tofu with spinach and tomato,Lentil soup with a side of whole-wheat bread,Lentil soup with whole-wheat bread,Mixed nuts and dried fruit
6,Sunday,Tofu scramble with veggies,Lentil salad with mixed greens,Chickpea and vegetable curry with brown rice,Apple slices with almond butter
